In [1]:
import sys
import os
import random

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from matplotlib import pyplot as plt

import tensorflow as tf
import keras
from keras import backend as K
from keras import initializers
from keras.layers import Dense, Input, Activation, multiply, Lambda
from keras.models import Sequential, Model, load_model
from keras.layers.merge import add, concatenate

from deap import base, creator, tools, algorithms
from multiprocessing import Pool
from scoop import futures

import copy
import pickle

Using TensorFlow backend.


In [2]:
act_dict = {0: 'linear', 1: 'multiply', 2: 'inverse', 3: 'squared', 4: 'sqrt'}
np.random.seed(1000)
weight_dict = {0: 0, 1: 1, 2: np.random.uniform(0,0.001,1)[0]}
bias_dict = {0: 0, 1: np.random.uniform(0,0.001,1)[0]}
print(weight_dict)
print(bias_dict)
nlayers = 4
nNodes = [5, 5, 3, 1]
nact_terms = sum(nNodes[1:])
nweight_terms = sum([nNodes[i-1]*nNodes[i] for i in range(1, nlayers)])
nbias_terms = nact_terms
p = 0.1
save_weights = False
save_weights_mutation = False
save_weights_crossover = False
Load_Old_Population = False

{0: 0, 1: 1, 2: 0.0006535895854646095}
{0: 0, 1: 0.00011500694312440574}


In [3]:
Individual_DB = []
MSE_Test_DB = []
Weight_Bias_DB = []

In [4]:
df = pd.read_csv("PropellantData_v3.csv")

In [5]:
inputs = np.array(df[['a','b', 'c', 'd', 'HeatofFormation']])
inputs = inputs.astype(np.float)
outputs = np.array(df['Isp']).reshape(-1, 1)
inputs = np.around(inputs, decimals=6)
outputs = np.around(outputs, decimals=2)

In [6]:
class CustomDense(keras.layers.Layer):
    def __init__(self, num_units, input_num, activation, name, trainable_weight, trainable_bias):
        super(CustomDense, self).__init__()
        self.num_units = num_units
        self.activation = Activation(activation)
        self.trainable_weight = trainable_weight
        self.trainable_bias = trainable_bias
        self.name = name
        name_w = 'w'+self.name[1:]
        name_b = 'b'+self.name[1:]
        self.weight = self.add_weight(shape=(input_num, self.num_units), name=name_w, trainable=self.trainable_weight, initializer="zeros")
        self.bias = self.add_weight(shape=(self.num_units,), name=name_b, trainable=self.trainable_bias, initializer="zeros")
        
    def call(self, input):
        y = tf.matmul(input, self.weight) + self.bias
        y = self.activation(y)
        return y

In [7]:
def f3(w):
    return w

In [8]:
class Individual_O:
    # Class to represent an individual
    def __init__(self, nLayers, nNodes, max1, max2, max3):
        self.nlayers = nLayers
        self.nNodes = nNodes
        self.nact_terms = sum(self.nNodes[1:])
        self.nweight_terms = sum([self.nNodes[i-1]*self.nNodes[i] for i in range(1, self.nlayers)])
        self.nbias_terms = self.nact_terms
        self.individual = None
        self.trainable = None
        self.weight_list = None
        self.bias_list = None
        self.weight_biases = None
        self.complexity = None
        self.MSE_Test = None
        self.Objective = None
        self.fitness = None
        self.model = None
        self.history = None
        self.epochs = None
        self.max1 = max1
        self.max2 = max2
        self.max3 = max3
    
    #Returns a copy of the individual
    def object_copy(self):
        cp = Individual_O(self.nlayers, self.nNodes, self.max1, self.max2, self.max3)
        cp.set_individual(copy.deepcopy(self.individual))
        cp.set_weight_bias(copy.deepcopy(self.weight_biases))
        return cp
        
        
    # Creates a specific individual
    def set_individual(self, individual):
        self.individual = individual
        self.calc_trainable()
        self.calc_complexity()
        self.set_random_weight_bias()
        
    def reset_model(self):
        self.Objective = None
        self.fitness = None
        self.weight_list = None
        self.bias_list = None
        self.calc_trainable()
        self.calc_complexity()
        
    # Creates a random individual
    def randomize_individual(self):
        x = []
        for l in range(1, self.nlayers):
            x.append([])
            for n in range(1, self.nNodes[l]+1):
                x[l-1].append([])
                x[l-1][n-1].append(random.randint(0, self.max1))
                for i in range(1, self.nNodes[l-1]+1):
                    x[l-1][n-1].append(random.randint(0, self.max2))
                x[l-1][n-1].append(random.randint(0, self.max3))
        self.individual = x
        self.calc_trainable()
        self.set_random_weight_bias()
        self.calc_complexity()
        
    # Creates trainable list
    def calc_trainable(self):
        trainable_list = []
        for l in range(1, self.nlayers):
            trainable_list.append([])
            for n in range(self.nNodes[l]):
                trainable_list[l-1].append([])
                for i in range(1, nNodes[l-1]+1):
                    if (self.individual[l-1][n][i] == 2):
                        trainable_list[l-1][n].append(True)
                    else:
                        trainable_list[l-1][n].append(False)
                if (self.individual[l-1][n][i+1] == 1):
                    trainable_list[l-1][n].append(True)
                else:
                    trainable_list[l-1][n].append(False)
        self.trainable = trainable_list
        
    # Calculates complexity of the model
    def calc_complexity(self):
        actfunc_term = 0
        wtbs_term = 0
        for l in range(1, self.nlayers):
            for n in range(self.nNodes[l]):
                actfunc_term += self.individual[l-1][n][0]
                wtbs_term += np.sum(self.individual[l-1][n][1:nNodes[l]])
                wtbs_term += np.sum(self.individual[l-1][n][nNodes[l]]*2)
        self.complexity = actfunc_term + wtbs_term
        
    # Randomizes all the weights and biases
    def set_random_weight_bias(self):
        x = []
        for l in range(1, self.nlayers):
            x.append([])
            for n in range(1, self.nNodes[l]+1):
                x[l-1].append([])
                for i in range(1, self.nNodes[l-1]+1):
                    x[l-1][n-1].append(weight_dict[self.individual[l-1][n-1][i]])
                x[l-1][n-1].append(bias_dict[self.individual[l-1][n-1][-1]])
        self.weight_biases = x
        
    def format_weight_bias(self, weight, bias):
        bias_count = 0
        weight_count = 0
        x = []
        for l in range(1, self.nlayers):
            x.append([])
            for n in range(1, self.nNodes[l]+1):
                x[l-1].append([])
                for i in range(1, self.nNodes[l-1]+1):
                    x[l-1][n-1].append(float(weight[weight_count]))
                    if ((i)%(self.nNodes[l-1])==0):
                        x[l-1][n-1].append(float(bias[bias_count]))
                    weight_count += 1
                    bias_count += 1
        self.weight_biases = x
        
    def set_weight_bias(self, weight_biases):
        self.weight_biases = weight_biases
        
    def update_weight_bias(self):
        #Update Weight/Bias List
        weight_list = []
        bias_list = []
        for i in range(0, sum(np.multiply(self.nNodes[:-1], self.nNodes[1:]))):
            weights = self.model.layers[i].get_weights()
            for weight in weights:
                if (weight.shape == (1,1)):
                    weight_list.append(weight)
                else:
                    bias_list.append(weight)
        self.weight_list = np.array(weight_list)
        self.bias_list = np.array(bias_list)
        
    # Builds the model
    def create_model(self, constraint_strength=10^3):
        x = self.individual
        self.weight_biases
        constraints = []
    
        inputs = []
        for i in range(self.nNodes[0]):
            inputs.append(Input(shape=(1,)))
    
        a = []
        for l in range(1, self.nlayers):
            a.append([])
            for n in range(1, self.nNodes[l]+1):
                a[l-1].append([])
                if l == 1:
                    a[l-1][n-1] = self.create_node(inputs, 'a' + str(l) + str(n), self.trainable[l-1][n-1], x[l-1][n-1], constraints)
                else:
                    a[l-1][n-1] = self.create_node(a[l-2], 'a' + str(l) + str(n), self.trainable[l-1][n-1], x[l-1][n-1], constraints)
    
    
        # make some adaptations depending on how many root activation functions are used
        if len(constraints) > 1:
            constraint = add(constraints)
        elif len(constraints) == 1:
            constraint = constraints[0]
        else:
            constraint = Lambda(lambda x: x-x)(a[0][0])
    
        a[-1].append(constraint)
        model = Model(inputs=inputs, outputs=a[-1])
    
        # Learning rate decay/schedule
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-1,
                                                                decay_steps=50000,
                                                                decay_rate=0.1,
                                                                staircase=False)
    
        # Optimizers
        #optimizer = tf.keras.optimizers.SGD(lr=1e-2)
        #optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
        #optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr_schedule)
        #optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr_schedule)
        #optimizer = tf.keras.optimizers.Adamax(learning_rate=lr_schedule)
        #optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-3)
        #optimizer = tf.keras.optimizers.Ftrl(lr=1e-3)
    
        model.compile(loss=['mse','mse'], loss_weights=[1, constraint_strength], optimizer=optimizer)
        model.layers.sort(key = lambda x: x.name, reverse=False)
    
        layer_list = []
        for i in range(len(model.layers)):
            name = model.layers[i].name
            if ( ("activation" in name) or ("input" in name) or ("add" in name) or ("multiply" in name) or ("lambda" in name)):
                continue
            else:
                layer_list.append(i)
    
        index = 0
        for l in range(1, self.nlayers):
            for n in range(1, self.nNodes[l]+1):
                for i in range(0, self.nNodes[l-1]):
                    if ((i+1)%(self.nNodes[l-1])==0):
                        if (model.layers[layer_list[index]].get_weights()[0].shape==(1,1)):
                            model.layers[layer_list[index]].set_weights( [ np.array( [[ self.weight_biases[l-1][n-1][i] ]] ),  np.array( [ self.weight_biases[l-1][n-1][i+1] ] ) ] )
                        else:
                            model.layers[layer_list[index]].set_weights( [ np.array( [ self.weight_biases[l-1][n-1][i+1] ] ),  np.array( [[ self.weight_biases[l-1][n-1][i] ]] ) ] )
                    else:
                        if (model.layers[layer_list[index]].get_weights()[0].shape==(1,1)):
                            model.layers[layer_list[index]].set_weights( [ np.array( [[ self.weight_biases[l-1][n-1][i] ]] ),  np.array( [ 0 ] ) ] )
                        else:
                            model.layers[layer_list[index]].set_weights( [ np.array( [ 0 ] ),  np.array( [[ self.weight_biases[l-1][n-1][i] ]] ) ] )
                    index += 1

        self.model = model    
    
    # Creates nodes
    def create_node(self, inputs, name, trainable, x_input, constraints):
        base = name
        act = act_dict[x_input[0]]
  
        an = []
        n = []
        for i in range(len(inputs)):
            n = base + str(i + 1)
            if i < len(inputs)-1:
                an.append(CustomDense(1, 1, activation = 'linear', name=n, trainable_weight=trainable[i], trainable_bias=False) (inputs[i]))
            else:
                an.append(CustomDense(1, 1, activation = 'linear', name=n, trainable_weight=trainable[i], trainable_bias=trainable[len(trainable)-1]) (inputs[i]))

        if (act == "multiply"):
            an = Activation(lambda x: self.custom_multiply(x, x_inputs=x_input[1:])) (an)
        else:
            an = add(an)
            if (act == "squared"):
                an = Activation(self.squared_act) (an)
            elif (act == "cubed"):
                an = Activation(self.cubed_act) (an)
            elif (act == "sqrt"):
                an = Activation(self.sqrt_act) (an)
            elif (act == "inverse"):
                an = Activation(self.inverse_act) (an)
            else:
                an = Activation(act) (an)
            
        # allow for negative square roots and 4th roots but push them to positive values during back propagation
        if act == 'sqrt' or act == '4th_rt':
            constraints.append(Activation('relu')(Lambda(lambda x: tf.negative(x))(an)))
        return an
    
    ## Activation functions
    
    # Multiply Activation
    def custom_multiply(self, x, x_inputs):
        activation_inputs = []
        zero_inputs = []
        for t in range(0, len(x)):
            if x_inputs[t] != 0:
                activation_inputs.append(x[t])
            elif (t == len(x)-1) & (x_inputs[len(x)] == 1):
                activation_inputs.append(x[t])
            else:
                zero_inputs.append(x[t])

        if activation_inputs == []:
            activation_tensor = 0
        elif len(activation_inputs) == 1:
            activation_tensor = activation_inputs[0]
        else:
            activation_tensor = multiply(activation_inputs)
        
        if zero_inputs == []:
            zero_tensor = 0
        elif len(zero_inputs) == 1:
            zero_tensor = zero_inputs[0]
        else:
            zero_tensor = multiply(zero_inputs)
        
        if not tf.is_tensor(zero_tensor):
            return activation_tensor
        elif not tf.is_tensor(activation_tensor):
            return zero_tensor
        else:
            return add([activation_tensor, zero_tensor])
        
    def inverse_act(self, x):
        return (1/x)
    
    def sqrt_act(self, x):
        return tf.sign(x)* tf.sqrt(tf.abs(x))
    
    def cubed_act(self, x):
        return x*x*x
    
    def squared_act(self, x):
        return x*x
       
    ## Training Functions
    def train(self, train_inputs, train_outputs, verbose=False):
        mae_es= keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000,
                                              min_delta=1e-8, verbose=0, mode='auto', restore_best_weights=True)
    
        terminate = keras.callbacks.TerminateOnNaN()

        EPOCHS = 50000 # Number of EPOCHS
        history = self.model.fit([train_inputs[:, i] for i in range(0, nNodes[0])], [train_outputs, np.zeros(train_outputs.shape)], epochs=EPOCHS,
                            shuffle=False, batch_size=32, verbose = 0, callbacks=[terminate, mae_es],
                            validation_split=0.3)
        # Test changing batch size
        if verbose:
            plt.figure()
            plt.xlabel('Epoch')
            plt.ylabel('Mean Sq Error')
            plt.plot(history.epoch, np.array(history.history['loss']),label='Training loss')
            plt.plot(history.epoch, np.array(history.history['val_loss']),label='Val loss')
            plt.legend()
            plt.show()
        self.history = history
    
    def cv_error(self, inputs, outputs, nSplits):
        splits = nSplits
        kf = KFold(n_splits=splits, shuffle=True, random_state=42)
        kf.get_n_splits(inputs)
        self.epochs = 0
    
        cv_mse_list = []
        cv_weight_list = []
        cv_bias_list = []

        for train_index, test_index in kf.split(inputs):
            self.create_model()
            if (any(self.trainable) == True):
                try:
                    trained = True
                    self.train(inputs[train_index, :], outputs[train_index], verbose=False)
                except:
                    trained = False
                    self.set_random_weight_bias()
            self.update_weight_bias()
            #handle nan weights and biases
            if (np.isnan(self.weight_list).any()):
                cv_mse = 1e50
                break
            elif (np.isnan(self.bias_list).any()):
                cv_mse = 1e50
                break
            elif not trained:
                cv_mse = 1e50
                break
            else:
                cv_mse = self.model.evaluate([inputs[test_index, i] for i in range(0, nNodes[0])], [outputs[test_index], np.zeros(outputs[test_index].shape)], verbose=0)
                if (np.isnan(cv_mse).any()):
                    cv_mse = 1e50
                    break
                elif cv_mse[0] == float('inf'):
                    cv_mse = 1e50
                    break
                else:
                    cv_mse = cv_mse[1]
                    cv_mse = np.around(cv_mse, decimals=6)
                    self.epochs += len(self.history.history['loss'])/nSplits
                    
            if cv_weight_list == []:
                cv_weight_list = self.weight_list
                cv_bias_list = self.bias_list
            else:
                cv_weight_list += self.weight_list
                cv_bias_list += self.bias_list
    
            cv_mse_list.append(cv_mse)
                
        if cv_mse == 1e50:
            self.MSE_Test = cv_mse
        else:
            self.MSE_Test = np.mean(cv_mse_list)
            self.format_weight_bias(np.array([weights/splits for weights in cv_weight_list]), np.array([bias/splits for bias in cv_bias_list]))
        self.model = None
        
    def objective_function(self, nSplits=5):
        if self.individual not in Individual_DB:
            self.cv_error(inputs, outputs, nSplits)
            self.history = None
            self.Objective = self.MSE_Test + p*self.complexity
            self.fitness = self.Objective
            
            print ("Individual: ", self.individual, flush=True)
            print ("Objective function: ", self.MSE_Test, self.complexity, self.Objective, self.gen, self.epochs, flush=True)
    
            Individual_DB.append(copy.deepcopy(self.individual))
            MSE_Test_DB.append(copy.deepcopy(self.MSE_Test))
            Weight_Bias_DB.append(copy.deepcopy(self.weight_biases))
                               
        else:
            self.MSE_Test = MSE_Test_DB[Individual_DB.index(self.individual)]
            self.Objective = self.MSE_Test + p*self.complexity
            self.fitness = self.Objective
            self.weight_biases = Weight_Bias_DB[Individual_DB.index(self.individual)]
                                        
        K.clear_session()

In [9]:
def init_repeat(nlayers, nNodes, max1=4, max2=2, max3=1):
    ind = Individual_O(nlayers, nNodes, max1, max2, max3)
    ind.randomize_individual()
    return ind

In [10]:
def custom_mutation(individual, indpb, max1, max2, max3):
    for l in range(1, individual.nlayers):
        for n in range(1, individual.nNodes[l]+1):
            if random.random() < indpb:
                individual.individual[l-1][n-1][0] = random.randint(0, max1)
                for i in range(1, individual.nNodes[l-1]+1):
                    individual.weight_biases[l-1][n-1][i-1] = weight_dict[individual.individual[l-1][n-1][i]]
                individual.weight_biases[l-1][n-1][i] = bias_dict[individual.individual[l-1][n-1][i+1]]
            for i in range(1, individual.nNodes[l-1]+1):
                if random.random() < indpb:
                    individual.individual[l-1][n-1][i] = random.randint(0, max2)
                    individual.weight_biases[l-1][n-1][i-1] = weight_dict[individual.individual[l-1][n-1][i]]
            if random.random() < indpb:
                individual.individual[l-1][n-1][i+1] = random.randint(0, max3)
                individual.weight_biases[l-1][n-1][i] = bias_dict[individual.individual[l-1][n-1][i+1]]
    if not save_weights:
        individual.set_random_weight_bias()
    elif not save_weights_mutation:
        individual.set_random_weight_bias()
        
    individual.reset_model()
    return individual,

In [11]:
def custom_crossover(individual1, individual2):
    LAYER = random.randint(1, nlayers-1)
    NODE = random.randint(0, nNodes[LAYER]-1)
    temp = copy.deepcopy(individual1.individual[LAYER-1][NODE])
    temp_weights = copy.deepcopy(individual1.weight_biases[LAYER-1][NODE])
    individual1.individual[LAYER-1][NODE] = copy.deepcopy(individual2.individual[LAYER-1][NODE])
    individual1.weight_biases[LAYER-1][NODE] = copy.deepcopy(individual2.weight_biases[LAYER-1][NODE])
    individual2.individual[LAYER-1][NODE] = copy.deepcopy(temp)
    individual2.weight_biases[LAYER-1][NODE] = copy.deepcopy(temp_weights)
    
    if not save_weights:
        individual1.set_random_weight_bias()
        individual2.set_random_weight_bias()
    elif not save_weights_crossover:
        weight1 = []
        weight2 = []
        for i in range(1,individual1.nNodes[LAYER-1]+1):
            weight1.append(weight_dict[individual1.individual[LAYER-1][NODE][i]])
            weight2.append(weight_dict[individual2.individual[LAYER-1][NODE][i]])
        weight1.append(bias_dict[individual1.individual[LAYER-1][NODE][i+1]])
        weight2.append(bias_dict[individual2.individual[LAYER-1][NODE][i+1]])
        
        individual1.weight_biases[LAYER-1][NODE] = weight1
        individual2.weight_biases[LAYER-1][NODE] = weight2
        
    individual1.reset_model()
    individual2.reset_model()
    return individual1, individual2

In [12]:
def objective_function(individual):
    individual.objective_function()
    return [individual.Objective, individual.weight_biases]

In [13]:
def custom_eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm reproduce the simplest evolutionary algorithm as
    presented in chapter 7 of [Back2000]_.

    :param population: A list of individuals.
    :param toolbox: A :class:`~deap.base.Toolbox` that contains the evolution
                    operators.
    :param cxpb: The probability of mating two individuals.
    :param mutpb: The probability of mutating an individual.
    :param ngen: The number of generation.
    :param stats: A :class:`~deap.tools.Statistics` object that is updated
                  inplace, optional.
    :param halloffame: A :class:`~deap.tools.HallOfFame` object that will
                       contain the best individuals, optional.
    :param verbose: Whether or not to log the statistics.
    :returns: The final population
    :returns: A class:`~deap.tools.Logbook` with the statistics of the
              evolution

    The algorithm takes in a population and evolves it in place using the
    :meth:`varAnd` method. It returns the optimized population and a
    :class:`~deap.tools.Logbook` with the statistics of the evolution. The
    logbook will contain the generation number, the number of evaluations for
    each generation and the statistics if a :class:`~deap.tools.Statistics` is
    given as argument. The *cxpb* and *mutpb* arguments are passed to the
    :func:`varAnd` function. The pseudocode goes as follow ::

        evaluate(population)
        for g in range(ngen):
            population = select(population, len(population))
            offspring = varAnd(population, toolbox, cxpb, mutpb)
            evaluate(offspring)
            population = offspring

    As stated in the pseudocode above, the algorithm goes as follow. First, it
    evaluates the individuals with an invalid fitness. Second, it enters the
    generational loop where the selection procedure is applied to entirely
    replace the parental population. The 1:1 replacement ratio of this
    algorithm **requires** the selection procedure to be stochastic and to
    select multiple times the same individual, for example,
    :func:`~deap.tools.selTournament` and :func:`~deap.tools.selRoulette`.
    Third, it applies the :func:`varAnd` function to produce the next
    generation population. Fourth, it evaluates the new individuals and
    compute the statistics on this population. Finally, when *ngen*
    generations are done, the algorithm returns a tuple with the final
    population and a :class:`~deap.tools.Logbook` of the evolution.

    .. note::

        Using a non-stochastic selection method will result in no selection as
        the operator selects *n* individuals from a pool of *n*.

    This function expects the :meth:`toolbox.mate`, :meth:`toolbox.mutate`,
    :meth:`toolbox.select` and :meth:`toolbox.evaluate` aliases to be
    registered in the toolbox.

    .. [Back2000] Back, Fogel and Michalewicz, "Evolutionary Computation 1 :
       Basic Algorithms and Operators", 2000.
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if ind.fitness == None]
    for ind in invalid_ind:
        ind.gen = 0
    data_stream = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, data in zip(invalid_ind, data_stream):
        ind.Objective = data[0]
        ind.fitness = data[0]
        ind.weight_biases = data[1]
    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        offspring = custom_varAnd(offspring, toolbox, cxpb, mutpb, gen, ngen)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if ind.Objective == None]
        for ind in invalid_ind:
            ind.gen = gen
        data_stream = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, data in zip(invalid_ind, data_stream):
            ind.Objective = data[0]
            ind.fitness = data[0]
            ind.weight_biases = data[1]

        # Replace the current population by the offspring
        population[:] = offspring
        
        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)
    return population, logbook

In [14]:
def custom_varAnd(population, toolbox, cxpb, mutpb, gen, ngen):
    """Part of an evolutionary algorithm applying only the variation part
    (crossover **and** mutation). The modified individuals have their
    fitness invalidated. The individuals are cloned so returned population is
    independent of the input population.

    :param population: A list of individuals to vary.
    :param toolbox: A :class:`~deap.base.Toolbox` that contains the evolution
                    operators.
    :param cxpb: The probability of mating two individuals.
    :param mutpb: The probability of mutating an individual.
    :returns: A list of varied individuals that are independent of their
              parents.

    The variation goes as follow. First, the parental population
    :math:`P_\mathrm{p}` is duplicated using the :meth:`toolbox.clone` method
    and the result is put into the offspring population :math:`P_\mathrm{o}`.  A
    first loop over :math:`P_\mathrm{o}` is executed to mate pairs of
    consecutive individuals. According to the crossover probability *cxpb*, the
    individuals :math:`\mathbf{x}_i` and :math:`\mathbf{x}_{i+1}` are mated
    using the :meth:`toolbox.mate` method. The resulting children
    :math:`\mathbf{y}_i` and :math:`\mathbf{y}_{i+1}` replace their respective
    parents in :math:`P_\mathrm{o}`. A second loop over the resulting
    :math:`P_\mathrm{o}` is executed to mutate every individual with a
    probability *mutpb*. When an individual is mutated it replaces its not
    mutated version in :math:`P_\mathrm{o}`. The resulting :math:`P_\mathrm{o}`
    is returned.

    This variation is named *And* because of its propensity to apply both
    crossover and mutation on the individuals. Note that both operators are
    not applied systematically, the resulting individuals can be generated from
    crossover only, mutation only, crossover and mutation, and reproduction
    according to the given probabilities. Both probabilities should be in
    :math:`[0, 1]`.
    """
    offspring = [ind.object_copy() for ind in population]
    #offspring = population
    
    # Apply crossover and mutation on the offspring
    for i in range(1, len(offspring), 2):
        if random.random() < cxpb:
            offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                          offspring[i])
    for i in range(len(offspring)):
        if random.random() < mutpb[(gen-1)//(ngen//len(mutpb))]:
            offspring[i], = toolbox.mutate(offspring[i], mutpb[(gen-1)//(ngen//len(mutpb))])
            
    return offspring

In [15]:
################### DEAP #####################
#create fitness class and individual class
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", Individual_O, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
#pool = Pool(1)
toolbox.register("map", futures.map)
#toolbox.register("attr_int", random.randint, 0, 3)

#gen = initRepeat(list, random.randint, 3, 7, 4)
toolbox.register("create_individual", init_repeat, 
                 nlayers, nNodes, max1=4, max2=2, max3=1)
toolbox.register("population", tools.initRepeat, list, toolbox.create_individual)

cxpb = 0
mutpb = [0, 0.8, 0.5, 0.2, 0.1]
#mutpb = 0.5
ngens = 50

toolbox.register("mate", custom_crossover)
#toolbox.register("mutate", tools.mutUniformInt, low=0, up=3, indpb=mutpb)
toolbox.register("mutate", custom_mutation, max1=4, max2=2, max3=1)
toolbox.register("select", tools.selTournament, tournsize=80)
toolbox.register("evaluate", objective_function)

def main():
    random.seed(10000)
    
    if not Load_Old_Population:
        population = toolbox.population(n=800)
    else:
        with open("P" + str(p) + "_1", 'rb') as f:
            population = pickle.load(f)
    stats = tools.Statistics(lambda ind: ind.Objective)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    pop, logbook = custom_eaSimple(population, toolbox, cxpb, mutpb, ngens, stats=stats, verbose=True)

    return pop, logbook

if __name__ == "__main__":
    pop, logbook = main()
    
    # Save final population
    filename = "P" + str(p)
    with open(filename, 'wb') as f:
        pickle.dump(pop, f)
        f.close()
        
    print (logbook, flush=True)


/home/nanohub/psalek/.local/lib/python3.7/site-packages/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().
  RuntimeWarning


Individual:  [[[4, 1, 0, 2, 0, 2, 0], [0, 1, 2, 1, 0, 0, 1], [0, 1, 0, 2, 0, 0, 1], [4, 2, 1, 2, 2, 0, 0], [0, 1, 0, 1, 2, 0, 0]], [[2, 1, 1, 1, 1, 1, 1], [4, 2, 2, 1, 2, 2, 0], [0, 1, 1, 2, 2, 2, 1]], [[0, 1, 0, 1, 0]]]
Objective function:  3.2685180000000003 57.0 8.968518 0 13.0


/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/ipykernel_launcher.py:361: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


Individual:  [[[1, 0, 1, 2, 2, 2, 0], [4, 1, 2, 2, 0, 2, 0], [4, 0, 1, 1, 1, 0, 1], [3, 1, 2, 2, 0, 2, 1], [4, 0, 2, 1, 0, 2, 0]], [[0, 0, 0, 0, 1, 0, 1], [3, 0, 0, 1, 0, 2, 1], [2, 1, 2, 1, 0, 1, 0]], [[2, 2, 2, 0, 0]]]
Objective function:  6.2547305 71.0 13.3547305 0 8.0
Batch 1: Invalid loss, terminating training
Batch 0: Invalid loss, terminating training
Individual:  [[[2, 1, 2, 2, 2, 1, 1], [3, 0, 2, 0, 0, 1, 1], [2, 1, 0, 2, 1, 2, 1], [1, 0, 0, 2, 1, 2, 0], [1, 1, 0, 1, 0, 2, 0]], [[2, 1, 2, 2, 2, 2, 0], [1, 1, 1, 1, 2, 2, 1], [3, 0, 2, 1, 2, 0, 1]], [[3, 0, 0, 2, 0]]]
Objective function:  1e+50 67.0 1e+50 0 0


/home/nanohub/psalek/.local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,activation_52_loss,lambda_1_loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


...

/home/nanohub/psalek/.local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,activation_108_loss,add_16_loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Individual:  [[[3, 1, 2, 2, 2, 2, 1], [4, 2, 2, 1, 2, 0, 1], [3, 2, 1, 0, 2, 0, 0], [3, 2, 2, 1, 0, 2, 1], [0, 1, 2, 2, 2, 2, 1]], [[4, 0, 0, 1, 2, 2, 1], [1, 2, 0, 0, 1, 1, 1], [4, 1, 2, 2, 1, 0, 1]], [[2, 1, 0, 2, 0]]]
Objective function:  5.790599 80.0 13.790599 19 7.0
19 	5     	4e+49      	13.7906	1e+50      
Individual:  [[[4, 1, 1, 1, 1, 2, 1], [4, 0, 0, 1, 2, 0, 1], [4, 2, 2, 1, 1, 0, 0], [1, 1, 2, 2, 2, 1, 0], [3, 1, 2, 1, 2, 0, 0]], [[4, 1, 1, 0, 2, 1, 0], [1, 2, 0, 1, 0, 1, 1], [3, 0, 1, 0, 0, 1, 1]], [[3, 0, 0, 0, 0]]]
Objective function:  6.5003174999999995 66.0 13.1003175 20 6.0
Individual:  [[[3, 0, 2, 2, 2, 2, 1], [1, 0, 2, 1, 1, 0, 1], [4, 1, 0, 0, 2, 0, 0], [0, 2, 2, 1, 0, 1, 0], [3, 2, 2, 1, 1, 0, 0]], [[0, 2, 0, 2, 1, 0, 1], [3, 2, 1, 0, 0, 2, 0], [4, 0, 0, 0, 2, 1, 1]], [[0, 2, 2, 0, 0]]]
Objective function:  208.4271015 61.0 214.5271015 20 7.0
Individual:  [[[1, 0, 1, 1, 1, 1, 1], [3, 0, 0, 2, 0, 2, 0], [4, 0, 1, 0, 0, 1, 0], [4, 0, 2, 1, 2, 0, 0], [0, 2, 2, 0, 2,

Objective function:  1e+50 59.0 1e+50 34 0
Batch 0: Invalid loss, terminating training
Individual:  [[[3, 1, 1, 0, 1, 2, 0], [2, 2, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 2, 0], [4, 0, 1, 1, 2, 1, 0], [3, 0, 1, 0, 0, 1, 0]], [[4, 2, 0, 1, 1, 0, 0], [0, 1, 0, 2, 0, 0, 1], [4, 1, 0, 0, 0, 0, 1]], [[0, 2, 1, 0, 1]]]
Objective function:  1e+50 58.0 1e+50 34 0
34 	5     	1e+50      	1e+50      	1e+50      
35 	5     	1e+50      	1e+50      	1e+50      


/home/nanohub/psalek/.local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,activation_55_loss,add_10_loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Batch 0: Invalid loss, terminating training
Individual:  [[[3, 1, 1, 2, 1, 2, 0], [2, 2, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 2, 1], [1, 1, 1, 1, 1, 1, 0], [3, 1, 1, 1, 0, 1, 0]], [[3, 2, 0, 1, 1, 1, 0], [0, 1, 0, 2, 0, 0, 1], [1, 1, 0, 2, 0, 0, 1]], [[0, 0, 1, 2, 1]]]
Objective function:  1e+50 56.0 1e+50 36 0
Individual:  [[[2, 0, 1, 0, 2, 2, 0], [4, 0, 0, 0, 1, 0, 0], [0, 0, 2, 0, 2, 2, 1], [3, 2, 2, 1, 2, 0, 0], [3, 1, 1, 1, 0, 2, 0]], [[0, 0, 1, 1, 1, 0, 0], [0, 2, 0, 2, 0, 1, 1], [4, 2, 2, 0, 0, 0, 1]], [[0, 1, 1, 2, 1]]]
Objective function:  86307.048828 61.0 86313.148828 36 7.0
36 	5     	8e+49      	86313.1    	1e+50      
Batch 0: Invalid loss, terminating training
Individual:  [[[0, 0, 1, 0, 1, 2, 1], [2, 2, 0, 0, 0, 0, 0], [0, 2, 2, 0, 1, 2, 1], [4, 2, 1, 1, 2, 1, 1], [4, 1, 1, 2, 0, 1, 1]], [[4, 0, 0, 1, 1, 1, 0], [0, 1, 1, 2, 0, 0, 1], [1, 1, 0, 0, 0, 0, 1]], [[0, 0, 1, 2, 1]]]
Objective function:  1e+50 55.0 1e+50 37 0
Batch 0: Invalid loss, terminating training
Individual:  